# Notebook para Definir um Universo de Discurso, Particionar em Funções de Pertinência e Realizar Fuzzificação de Amostras
Este notebook tem como objetivo explorar o universo de discurso, particionar o domínio em funções de pertinência e realizar fuzzificação de amostras. Ele também inclui uma análise comparativa textual dos resultados obtidos.

## 1. Definir Universo de Discurso
Defina o intervalo do universo de discurso e as amostras para fuzzificação. Exemplo:

In [ ]:
# Definição do universo de discurso
dominio = (0, 100)  # Intervalo do universo de discurso
samples = [25, 75]  # Amostras para fuzzificação

## 2. Particionar o Domínio em Funções de Pertinência
Implemente a geração de parâmetros para pelo menos quatro funções de pertinência uniformemente espaçadas para cada tipo implementado na Atividade 1.1.

In [ ]:
import numpy as np

# Função para gerar parâmetros para diferentes tipos de funções de pertinência
def generate_params(X, types, n):
    centers = np.linspace(X[0], X[1], n)  # Centros uniformemente distribuídos
    step = (X[1] - X[0]) / (n - 1) if n > 1 else (X[1] - X[0])  # Espaçamento entre os centros
    params = []

    for i, t in enumerate(types):
        if t == 'gaussian':
            sigma = step / 2  # Sigma proporcional ao espaçamento
            params.append([centers[i], sigma])
        elif t == 'triangular':
            a = max(X[0], centers[i] - step)  # Início da base
            b = centers[i]                   # Pico
            c = min(X[1], centers[i] + step)  # Fim da base
            params.append([a, b, c])
        elif t == 'trapezoidal':
            a = max(X[0], centers[i] - step)  # Início da base
            b = max(X[0], centers[i] - step / 2)  # Início do topo
            c = min(X[1], centers[i] + step / 2)  # Fim do topo
            d = min(X[1], centers[i] + step)  # Fim da base
            params.append([a, b, c, d])
        elif t == 'sigmoidal':
            a = 1  # Inclinação padrão
            c = centers[i]  # Centro
            params.append([a, c])
        else:
            raise ValueError(f"Tipo de função '{t}' não suportado!")
    
    return params

# Exemplo de tipos de funções e geração de parâmetros
types = ['gaussian', 'triangular', 'trapezoidal', 'sigmoidal']
n = len(types)
params = generate_params(dominio, types, n)
print("Parâmetros gerados:", params)

## 3. Fuzzificação de Duas Amostras
Calcule o grau de ativação de cada função de pertinência para as amostras fornecidas. Use as funções implementadas na Atividade 1.1.

In [ ]:
# Funções de pertinência implementadas
def gaussian(x, c, sigma):
    return np.exp(-0.5 * ((x - c) / sigma) ** 2)

def triangular(x, a, b, c):
    if a <= x < b:
        return (x - a) / (b - a)
    elif b <= x < c:
        return (c - x) / (c - b)
    else:
        return 0

def trapezoidal(x, a, b, c, d):
    if a <= x < b:
        return (x - a) / (b - a)
    elif b <= x <= c:
        return 1
    elif c < x <= d:
        return (d - x) / (d - c)
    else:
        return 0

def sigmoidal(x, a, c):
    return 1 / (1 + np.exp(-a * (x - c)))

# Cálculo dos graus de ativação
activations = []
for sample in samples:
    sample_activations = []
    for func_type, func_params in zip(types, params):
        if func_type == 'gaussian':
            activation = gaussian(sample, *func_params)
        elif func_type == 'triangular':
            activation = triangular(sample, *func_params)
        elif func_type == 'trapezoidal':
            activation = trapezoidal(sample, *func_params)
        elif func_type == 'sigmoidal':
            activation = sigmoidal(sample, *func_params)
        else:
            raise ValueError(f"Tipo de função desconhecido: {func_type}")
        sample_activations.append(activation)
    activations.append(sample_activations)

print("Ativações calculadas:", activations)

## 4. Plotar Funções de Pertinência e Ativações
Crie gráficos para cada tipo de função de pertinência mostrando o universo de discurso, as funções e as ativações das amostras.

In [ ]:
import matplotlib.pyplot as plt

# Função para plotar funções de pertinência e ativações
def plot_membership_with_samples(x, results, labels, samples, activations):
    plt.figure(figsize=(10, 6))
    for i, result in enumerate(results):
        plt.plot(x, result, label=f"{labels[i]}")
    for sample, sample_activations in zip(samples, activations):
        for i, activation in enumerate(sample_activations):
            plt.scatter(sample, activation, label=f"Sample {sample} - {labels[i]}: {activation:.2f}")
    plt.title("Funções de Pertinência e Ativações")
    plt.xlabel("x")
    plt.ylabel("Grau de Pertinência")
    plt.legend()
    plt.grid()
    plt.show()

# Geração de valores para o domínio
x = np.linspace(dominio[0], dominio[1], 100)
results = []
for func_type, func_params in zip(types, params):
    if func_type == 'gaussian':
        results.append([gaussian(val, *func_params) for val in x])
    elif func_type == 'triangular':
        results.append([triangular(val, *func_params) for val in x])
    elif func_type == 'trapezoidal':
        results.append([trapezoidal(val, *func_params) for val in x])
    elif func_type == 'sigmoidal':
        results.append([sigmoidal(val, *func_params) for val in x])

# Plotagem
plot_membership_with_samples(x, results, types, samples, activations)

## 5. Análise Textual Comparativa
Escreva uma análise textual comparando os resultados das ativações entre diferentes tipos de funções de pertinência, destacando suavidade e sensibilidade.

As funções de pertinência apresentaram comportamentos distintos em relação às amostras fornecidas:
- **Gaussian**: Apresenta suavidade e transições graduais, sendo menos sensível a mudanças bruscas.
- **Triangular**: Possui transições lineares, com maior sensibilidade em regiões próximas aos vértices.
- **Trapezoidal**: Combina regiões de transição linear com uma região plana, sendo útil para representar intervalos constantes.
- **Sigmoidal**: Apresenta uma curva suave e assimétrica, útil para modelar transições rápidas em uma direção específica.

Essas diferenças destacam a importância de escolher a função de pertinência adequada para cada aplicação.